### 节点预测任务实践

在此小节我们将利用的Planetoid的PubMed数据集, 来实践节点预测与边预测任务。

- [节点预测任务详解](https://zhuanlan.zhihu.com/p/427732420)

### 定义GAT

- 定义可以通过使用参数确定 GATConv 层数和 out_channel 的网络

In [2]:
import torch.nn as nn
import torch
import torch.functional as F
from torch_geometric.nn import GATConv, Sequential
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

torch.__version__

'1.10.2'

In [ ]:
class GAT(nn.Module):
    def __init__(self, input_features, num_classes, hidden_channels_list):
        """
        :param input_features: 数据集的特征数量
        :param num_classes: 图的类别数
        :param hidden_channels:
        """
        super(GAT, self).__init__()
        # 设置随机数种子
        torch.manual_seed(12345)
        # 拼接输入特征数和隐藏层数 : [input_features, hidden_channel , ...]
        hns = [input_features] + hidden_channels_list
        conv_list = []
        #
        for idx in range(len(hidden_channels_list)):
            # [input_features, hidden_channels_1]
            # [hidden_channels_1, hidden_channels_2]
            # ...
            # [hidden_channels_n-1, hidden_channels_n]
            conv_list.append((GATConv(in_channels=hns[idx], out_channels=hns[idx+1]),'x, edge_index -> x'))

        # 整合 多层网络
        self.conv_seq = Sequential('x, edge_index', conv_list)
        # linear
        self.linear = nn.Linear(hidden_channels_list[-1], num_classes) # [input_features, num_classes]

    def forward(self, x, edge_index):
        x = self.conv_seq(x, edge_index)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.linear(x)
        return x